In [ ]:
# v1
# update file
import re
import json
from google import genai
from google.genai import types

system_prompt = """Translate texts from English to Vietnamese.
The subject matter is Early Buddhism. Translations should be *easy to understand, using simple, modern, everyday language*.

RULE:
1. Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

2. Maintain the original paragraph structure; do not add or remove any text.

3. Key Term Explanation, explaining terms that are central to Early Buddhist thought and likely unfamiliar to a 10th-grader. won't explain every single
Pali word, only the ones that are crucial for understanding.
When translation, if key terms or concepts that require clarification then add explanations, add explanations will be short and to the point, avoiding philosophical debates.
Example, If the text says "the cessation of craving," I might translate it as: "sự chấm dứt tham ái (craving - sự ham muốn mạnh mẽ)."
But if it says "suffering," I likely won't add an explanation unless the context makes it unclear, because "khổ" is a common word.

4. Glossary (Strict Format): In the end, add The glossary, glossary will only include terms that were actually used and explained within the translated text itself.
**Format:** `Vietnamese Term / Pali Term / English Term: Expanded explanation from the in-text definition.`
No generated content: The explanation in Vietnamese must be consistent to the meaning in the text, no unrelated content.

```

<!--pg-->
Từ ngữ:

- con đường tám nhánh / aṭṭhaṅgika magga / eightfold path: Tám phương diện tu tập để đạt đến giác ngộ.
- giác ngộ / bodhi / enlightenment: Trạng thái hiểu biết hoàn toàn về sự thật.
- thiền định / samādhi / concentration: Tập trung tâm trí vào một đối tượng duy nhất.
"""

system_prompt2 = """Translate the following English text about Early Buddhism into Vietnamese.
The translation should be suitable for a 10th-grade reading level, using simple, modern, and everyday language.

RULE:

- Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

- **Paragraph Structure:** Maintain the original paragraph structure. Do not add or remove any text.

- **Readability:** Use clear, simple Vietnamese. Avoid complex or archaic vocabulary.

- **Key Term Explanations (IN-TEXT):**  Identify terms *crucial* to understanding Early Buddhist thought that a 10th-grader likely wouldn't know. Explain these terms *within the translation itself*, immediately after their first use. Keep explanations short and to the point.
    *   **Format:** `Vietnamese Term (Pali Term - English Term: short definition)`
    *   **Example:** Instead of just "cessation of craving," translate it as:  `sự chấm dứt tham ái (taṇhā - craving: sự ham muốn mãnh liệt, dai dẳng).`
    *   **Do NOT explain very common words** like "suffering" (`khổ`) unless the context is unusual. Only explain terms related to early Buddhism context.

- **Glossary (Strict Format):**
    *   **AFTER** the translation, create a glossary.
    *   **ONLY include terms explained IN-TEXT (Rule 3).**
    *   **Format:** `Vietnamese Term / Pali Term / English Term: Expanded explanation from the in-text definition.`

**Glossary Example (Following a Hypothetical Translation):**

<!--pg-->
Từ ngữ:

- sự chấm dứt tham ái / taṇhā / craving: Sự ham muốn mãnh liệt, dai dẳng, nguyên nhân chính của khổ đau.
- bốn sự thật cao quý / cattāri ariya saccāni / four noble truths: Bốn sự thật mà Đức Phật đã giác ngộ, nền tảng của Phật giáo.
- con đường tám nhánh / aṭṭhaṅgika magga / eightfold path: Tám phương diện tu tập để đạt đến giác ngộ.
- giác ngộ / bodhi / enlightenment: Trạng thái hiểu biết hoàn toàn về sự thật.
- thiền định / samādhi / concentration: Tập trung tâm trí vào một đối tượng duy nhất.
"""

system_prompt3 = """Translate content from English to Vietnamese, focusing on Theravada Buddhist texts. Use simple,
understandable, modern Vietnamese terminology for accessibility. For specialized Buddhist terms that are uncommon in
everyday language (such as "attachment" or "complete nirvana" or "purification of doubt"), please provide additional
explanation by including direct English and raw-explaination in vietnam translation in parentheses but only for the first occurrence of each term (explain once).

Example format: "thanh tịnh bằng tri kiến về đạo lộ (purification by knowledge and vision of the way - sự thanh lọc bằng kiến ​​thức và tầm nhìn về đường đạo)"

At the end of each translation, please include a glossary section with the following format:

<!--pg-->
Từ ngữ:
- Vietnamese term / Pali term / English term: Comprehensive explanation in Vietnamese
Example:
- bốn sự thật cao quý / cattāri ariya saccāni / four noble truths: Bốn sự thật mà Đức Phật đã giác ngộ, nền tảng của Phật giáo.

- Always maintain the original meaning while making the translation accessible to modern Vietnamese readers.
When encountering ambiguous terms, provide multiple possible interpretations if necessary.
For quoted passages, preserve the poetic quality where possible while ensuring clarity.
Focus particularly on accuracy with technical dharma terms and use simple, understandable, modern Vietnamese terminology for accessibility.
Maintain the original paragraph structure.
"""
# tempature = 0.5
system_prompt_simple = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.
- For rare or technical Buddhist terms,  translate use clear, modern, and accessible vietnamese words and provide a one-time explanation upon first use, including:
1. The original English term
2. A classical Vietnamese equivalent (if available)
3. A short, clear explanation (if needed)
Example: *sự vô tâm* (heedlessness - phóng dật); *hướng tâm đúng pháp* (careful attention-như lý tác ý-"Như lý": theo đúng lý, "tác ý": hướng tâm đến, suy nghĩ).

- Do not add any introduction or conclusion-translate only.
"""
system_prompt_simple = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.

- For *rare* or *technical Buddhist terms* or *keyword of sentence*,  translate use clear, modern, and accessible vietnamese words and provide a one-time explanation upon first use, including: (eg: vẻ đẹp)
1. The original English term (eg: feature of beauty)
2. A classical Vietnamese equivalent (if available) (eg: tướng khả ái)
3. A short, clear explanation (if needed)
Example 1: *sự vô tâm* (heedlessness - phóng dật); *hướng tâm đúng/sai pháp* (apply the mind irrationally/rationally-không/như lý tác ý-"như lý": theo đúng lý, "tác ý": hướng tâm đến, suy nghĩ).
Example 2: *buồn ngủ và uể oải* (dullness and drowsiness - hôn trầm và thụy miên).

- Do not add any introduction or conclusion-translate only.

-  Pre-defined translation sentence  :
1. I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty -> Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì tăng trưởng.
2. I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it -> Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt

Example:

The Chapter on Giving Up the Hindrances

“Mendicants, I do not see a single thing that gives rise to sensual
desire, or, when it has arisen, makes it increase and grow like the
feature of beauty. When you apply the mind irrationally to the
feature of beauty, sensual desire arises, and once arisen it increases
and grows.”
->
Chương về Từ Bỏ Các Trở Ngại (Hindrances - Triền Cái)
Này các Tỳ-kheo, Ta không thấy một pháp nào khác làm ham muốn dục tham (sensual desire -  kāma-rāga - dục tham : lòng ham muốn vào các đối tượng giác quan như sắc, thanh, hương, vị, xúc,  ý tưởng dễ chịu), chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh. Đó là *vẻ đẹp* (feature of beauty - subha nimitta - tướng khả ái). Khi các thầy hướng tâm sai pháp (apply the mind irrationally-không như lý tác ý-"như lý": theo đúng lý, "tác ý": hướng tâm đến, suy nghĩ). đến vẻ đẹp, dục tham chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh.
"""
# tempurature = 0.5
system_prompt_simple2 = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.

- For *rare* or *technical Buddhist terms* or *keyword of sentence*, translate use clear, modern, and accessible vietnamese words and provide a one-time explanation upon first use, including canonical Vietnamese terms for familiar readers.
Format: *modern, and accessible vietnamese terms*(orgin-english terms / canonical Vietnamese terms)
eg1: feature of beauty->*vẻ đẹp* (feature of beauty / tịnh tướng);
eg2: apply the mind irrationally/rationally->*hướng tâm đúng/sai pháp* (apply the mind irrationally/rationally / không / như lý tác ý);
eg3: dullness and drowsiness->*sự buồn ngủ và tâm trí lờ đờ* (dullness and drowsiness - thīna-middha - hôn trầm và thụy miên)
eg4: restlessness and remorse->*bồn chồn và hối hận* (restlessness and remorse / uddhacca-kukkucca - trạo cử và hối quá)

-  Pre-defined translation sentence  :
1. I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty -> Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì tăng trưởng.
2. I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it -> Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt

At the end of translation, please include a glossary section for The explanation should be clear and simple enough for a 5th grade reader to understand, not confusing.
With the following format:
<!--pg-->
Từ ngữ:
- Vietnamese term / English term / Pali term : Comprehensive explanation in Vietnamese
Example: Sân hận / ill will / byāpāda/ sân: sự tức giận, căm ghét, ác ý. Sân hận là một trong 5 triều cái (dục, sân, hôn trầm-thùy miên, trạo cử-hối quá, nghi ngờ)

- Do not add any introduction or conclusion-translate only.

---Full Example---:
INPUT:
# The Chapter on Giving Up the Hindrances

<!--pg-->
“1\. Mendicants, I do not see a single thing that gives rise to sensual
desire, or, when it has arisen, makes it increase and grow like the
feature of beauty. When you apply the mind irrationally to the
feature of beauty, sensual desire arises, and once arisen it increases
and grows.”

EXPECTED OUTPUT:
# Chương về Từ Bỏ Các Trở Ngại (Hindrances - Triền Cái)

<!--pg-->
1\. Này các Tỳ-kheo, Ta không thấy một pháp nào khác làm *sự ham muốn* (sensual desire/tham dục) chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh. Đó là *vẻ đẹp* (feature of beauty/tướng khả ái). Khi các thầy hướng tâm sai pháp (apply the mind irrationally/không như lý tác ý) đến vẻ đẹp, dục tham chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh.

<!--pg-->
Từ ngữ:
- Hướng tâm sai pháp/apply the mind irrationally/ayoniso manasikāra/không như lý tác ý: "Như lý" là "đúng như chân lý", "tác ý" là "tạo ý",
cả câu là "điều khiển suy nghĩ theo đúng thiện pháp"
"""

system_prompt_simple2 = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.
<RULEs>
1. For *rare* or *technical Buddhist terms* translation use clear, modern, and accessible Vietnamese words and provide a one-time explanation upon first use, including canonical Vietnamese terms for familiar readers.
Format: *modern, and accessible Vietnamese terms*(origin-english terms / canonical Vietnamese terms)
eg1: feature of beauty -> *vẻ đẹp* (feature of beauty / tịnh tướng).
eg2: apply the mind irrationally/rationally ->*hướng tâm đúng/sai pháp* (apply the mind irrationally/rationally / không/như lý tác ý).
eg3: dullness and drowsiness ->*sự buồn ngủ và tâm trí lờ đờ* (dullness and drowsiness / hôn trầm và thụy miên).
eg4: restlessness and remorse ->*bồn chồn và hối hận* (restlessness and remorse / trạo cử và hối quá).
eg5: unmindful and careless -> *thiếu thận trọng và bất cẩn* (unmindful and careless / thất niệm và buông lung)

2. Pre-defined translation sentence  :
   1. `I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty` -> `Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì tăng trưởng`.
   2. `I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it` -> `Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt`

3. At the end of translation, please include a glossary section (max 10 items), for explanation should be clear and simple enough for a 5th grade reader to understand, not. confusing.
With the following format:
<!--pg-->
Từ ngữ:
- Vietnamese term / English term / Pali term : Comprehensive explanation in Vietnamese
Example: Sân hận / ill will / byāpāda/ sân: sự tức giận, căm ghét, ác ý. Sân hận là một trong 5 triều cái (dục, sân, hôn trầm-thùy miên, trạo cử-hối quá, nghi ngờ)

4. Do not add any introduction or conclusion-translate only.
</RULEs>
<Full_Example>
<INPUT>
# The Chapter on Giving Up the Hindrances

<!--pg-->
“1\. Mendicants, I do not see a single thing that gives rise to sensual
desire, or, when it has arisen, makes it increase and grow like the
feature of beauty. When you apply the mind irrationally to the
feature of beauty, sensual desire arises, and once arisen it increases
and grows.”
</INPUT>
<OUTPUT>
# Chương về Từ Bỏ Các Trở Ngại (Hindrances - Triền Cái)

<!--pg-->
1\. Này các Tỳ-kheo, Ta không thấy một pháp nào khác làm *sự ham muốn* (sensual desire/tham dục) chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh. Đó là *vẻ đẹp* (feature of beauty/tướng khả ái). Khi các thầy hướng tâm sai pháp (apply the mind irrationally/không như lý tác ý) đến vẻ đẹp, dục tham chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh.

<!--pg-->
Từ ngữ:
- Hướng tâm sai pháp/apply the mind irrationally/ayoniso manasikāra/không như lý tác ý: "Như lý" là "đúng như chân lý", "tác ý" là "tạo ý",
cả câu là "điều khiển suy nghĩ theo đúng thiện pháp"
</OUTPUT>
</Full_Example>
"""
system_prompt_simple_no_glossary1 = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.
<RULEs>
1. For *rare* or *technical Buddhist terms* translation use clear, modern, and accessible Vietnamese words and provide a one-time explanation upon first use, including canonical Vietnamese terms for familiar readers.
Format: *modern, and accessible Vietnamese terms*(origin-english terms / canonical Vietnamese terms)
eg1: feature of beauty -> *vẻ đẹp* (feature of beauty / tịnh tướng).
eg2: apply the mind irrationally/rationally ->*hướng tâm đúng/sai pháp* (apply the mind irrationally/rationally / không/như lý tác ý).
eg3: dullness and drowsiness ->*sự buồn ngủ và tâm trí lờ đờ* (dullness and drowsiness / hôn trầm và thụy miên).
eg4: restlessness and remorse ->*bồn chồn và hối hận* (restlessness and remorse / trạo cử và hối quá).
eg5: unmindful and careless -> *thiếu thận trọng và bất cẩn* (unmindful and careless / thất niệm và buông lung)

2. Pre-defined translation sentence  :
   1. `I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty` -> `Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì tăng trưởng`.
   2. `I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it` -> `Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt`

4. Do not add any introduction or conclusion-translate only.
</RULEs>
<Full_Example>
<INPUT>
# The Chapter on Giving Up the Hindrances

<!--pg-->
“1\. Mendicants, I do not see a single thing that gives rise to sensual
desire, or, when it has arisen, makes it increase and grow like the
feature of beauty. When you apply the mind irrationally to the
feature of beauty, sensual desire arises, and once arisen it increases
and grows.”
</INPUT>
<OUTPUT>
# Chương về Từ Bỏ Các Trở Ngại (Hindrances - Triền Cái)

<!--pg-->
1\. Này các Tỳ-kheo, Ta không thấy một pháp nào khác làm *sự ham muốn* (sensual desire/tham dục) chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh. Đó là *vẻ đẹp* (feature of beauty/tướng khả ái). Khi các thầy hướng tâm sai pháp (apply the mind irrationally/không như lý tác ý) đến vẻ đẹp, dục tham chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh.
</OUTPUT>
</Full_Example>
"""

system_prompt_simple_no_glossary = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.
<RULEs>
1. ONLY for *rate/technical Buddhist terms* translation use clear, modern, and accessible Vietnamese terms and provide a one-time explanation upon first use.
Format: *clear, modern, and accessible Vietnamese terms* (origin-english terms / modern and accessible Vietnamese translation)
eg1: feature of beauty -> *tịnh tướng* (feature of beauty / vẻ đẹp).
eg2: apply the mind irrationally/rationally ->*không/như lý tác ý* (apply the mind irrationally/rationally / hướng tâm đúng/sai pháp).
eg3: dullness and drowsiness ->*hôn trầm và thụy miên* (dullness and drowsiness / sự buồn ngủ và tâm trí lờ đờ).
eg4: restlessness and remorse ->*trạo cử và hối quá* (restlessness and remorse / bồn chồn và hối hận).
eg5: unmindful and careless -> *thất niệm và buông lung* (unmindful and careless / thiếu thận trọng và bất cẩn)
eg6: perceptions ->*nhận thức* (perceptions / tưởng, nhận ra).

2. Pre-defined translation sentence  :
   1. `I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty` -> `Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì tăng trưởng`.
   2. `I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it` -> `Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt`
3. With the poem (begin with `>` character):
  - 1. do not adding explanation.
  - 2. add `\` like origin-text.
eg:
input:
```
> yoked also to views,\
> and governed by ignorance,
>
> by those energetic, steadfast, and resolute,\
> practicing absorption, mindful, with guarded senses.
```
output:
```
> Cũng bị trói buộc bởi tà kiến,\
> Và bị vô minh chi phối,
>
> Bởi những người tinh tấn, kiên trì, quyết tâm,\
> Hành thiền, chánh niệm, thu thúc các căn.
```
4. Do not add any introduction or conclusion-translate only.

5. The following words should be used instead of Hán-Vietnamese words
Hán-Vietnamese | Vietnamese
Quý = Sợ điều ác
Tàm = Xấu hổ
Tín = Niềm tin
Tấn = Siêng năng
Tuệ = Trí tuệ
Niệm = Tỉnh thức
Hôn trầm = Uể oải sao nhãng
Thùy miên = Buồn ngủ
Sân hận = Tức giận
Trạo cử = Bồn chồn lo lắng
Hối quá = Hối hận
Lạc = An vui
Hỷ = Vui vẻ
Giới cấm thủ = Sùng bái giáo điều
Tùy niệm = Cần phải nhớ
Phóng dật = sao nhãng

6. Use correct Vietnamese grammar eg:
Tín lực = lực niềm tin
Vô tàm = không xấu hổ
tăng thượng tuệ = trí tuệ cao hơn

</RULEs>
"""

system_prompt_note = """Translate texts from English to Vietnamese.
The subject matter is Early Buddhism. Translations should be *easy to understand, using simple, modern, everyday language*.

RULE:
1. Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

2. Maintain the original paragraph structure; do not add or remove any text.

3. When translation, if key terms or concepts that require clarification, please provide the Vietnamese translation,
followed by the original English word in (), and a brief explanation if necessary.
Example1: `giới cấm thủ (adherence to rules and observances-sự tuân thủ các quy tắc và việc thực hiện các nghi lễ hoặc phong tục)`
Example2: `sân hận (ill will)`
"""

response = None
client = None
def translate_markdown(input_file: str, output_file: str, api_key: str, prompt, model, log, temperature=0):
    """
    Translates an English markdown file to Vietnamese using Google GenAI.

    Args:
        input_file: Path to the input English markdown file (e.g., "abc.md").
        output_file: Path to the output Vietnamese markdown file (e.g., "abc.vn.md").
        api_key: Your Google GenAI API key.
    """

    global response
    global client

    client = genai.Client(api_key=api_key)

    file = client.files.upload(file=input_file)
    # --- Core Translation Logic ---

    try:
        err = ""
        response = client.models.generate_content(
            model= model,
            contents=[file],
            config=types.GenerateContentConfig(
                system_instruction=prompt,
                # max_output_tokens=50000,
                # candidates_token_count = 20000,
                # input_token_limit = 20000,
                temperature=temperature
                # temperature=1
                ),
        )

        if response.candidates:
            if response.candidates[0].finish_reason != types.FinishReason.STOP:
                print(f": {input_file} resonse: {response.candidates[0].finish_reason}")
                # log.write(f"{filename}, exception: File not found\n")
                log.write(f": {input_file} resonse {response.candidates[0].finish_reason}")
                #If it is not stop, consider reducing the chunk size
        else: #No candidated return
            pass
            # err = "response.candidates is empty. reduce the chunk size"


        vietnamese_markdown = response.text

    except Exception as e:
      log.write(f'f"Error during translation: {e}')
      print(f'f"Error during translation: {e}')
    finally:
         # --- Output Handling ---
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(vietnamese_markdown)
                log.write(f'{input_file} ok')
        except Exception as e:
            log.write(f'{input_file} Error writing output file: {e}')


    # return f'{input_file} {output_file}:{err}'


import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# peterontech key

_api_key = os.environ.get("GOOGLE_API_KEY")
_temperature = 0.45

_model = "gemini-2.5-pro-preview-03-25"
# model ="gemini-2.0-flash-thinking-exp-01-21"
# model = "gemini-2.0-flash"
# _model="gemini-2.0-flash"
# _model = "gemini-2.5-flash-preview-04-17"
# _prompt = system_prompt3
_prompt = system_prompt_simple_no_glossary
# "Translate texts from English to Vietnamese. Translations should be *easy to understand, using simple, modern, everyday language*"


In [6]:

# trans folder

def trans_folder(infilelist, output_folder, limit=None, test=False):
    fi = open("tran_ressult.txt", "a")

    count = 1
    for filepath in infilelist: # os.listdir(input_folder):
        filename = os.path.basename(filepath)

        if limit is not None and count >= limit:
            break  # Stop if the limit is reached

        # input_filename = os.path.join(input_folder, filename)
        output_filename = os.path.join(output_folder, filename)

        print(f"Translating {filepath} to {output_filename}")

        try:
            # pass
            if test is False:
                translate_markdown(filepath, output_filename, _api_key, _prompt, _model, fi)
            else:
                print(filepath)
            count += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")  # Handle potential errors
    fi.close()

filelist = [

 '../docs/kinhtangchi/sujato-en/03/06-001-the-chapter-on-worthy-of-offerings.md',
 '../docs/kinhtangchi/sujato-en/03/06-002-the-chapter-on-warm-hearted.md',
 '../docs/kinhtangchi/sujato-en/03/06-003-the-chapter-on-unsurpassable.md',
 '../docs/kinhtangchi/sujato-en/03/06-004-the-chapter-on-deities.md',
 '../docs/kinhtangchi/sujato-en/03/06-005-the-chapter-with-dhammika.md',
 '../docs/kinhtangchi/sujato-en/03/06-006-the-great-chapter.md',
 '../docs/kinhtangchi/sujato-en/03/06-007-the-chapter-on-deities.md',
 '../docs/kinhtangchi/sujato-en/03/06-008-the-chapter-on-perfection.md',
 '../docs/kinhtangchi/sujato-en/03/06-009-the-chapter-on-coolness.md',
 '../docs/kinhtangchi/sujato-en/03/06-010-the-chapter-on-benefit.md',
 '../docs/kinhtangchi/sujato-en/03/06-011-the-chapter-on-triads.md',
 '../docs/kinhtangchi/sujato-en/03/06-012-the-chapter-on-the-ascetic-life.md',

]

filelist = [ filelist[11] ]
out_folder = '../docs/kinhtangchi/sujato-vi/03/'
# for filename in sorted([f for f in os.listdir(in_folder) if os.path.isfile(os.path.join(in_folder, f))]):
#     filelist.append(f'{in_folder}{filename}')
# filelist
# filelist = filelist[1:]
trans_folder(infilelist=filelist, output_folder=out_folder)

Translating ../docs/kinhtangchi/sujato-en/03/06-012-the-chapter-on-the-ascetic-life.md to ../docs/kinhtangchi/sujato-vi/03/06-012-the-chapter-on-the-ascetic-life.md


In [ ]:
# get all file
import os
filelist = []
# for filename in sorted(os.listdir('../docs/kinhtrungbo/nanamoli-bodhi-en')):
#     filelist.append(filename)

for filename in sorted([f for f in os.listdir('../docs/kinhtangchi/sujato-vi/03') if os.path.isfile(os.path.join('../docs/kinhtangchi/sujato-vi/03', f))]):
    filelist.append('../docs/kinhtangchi/sujato-en/03/'+filename)

filelist